# Pour P., lister les limites de qualités dans la data
***
_Les parties indispensables à run sont indiquées avec la mention [TO RUN], afin de reproduire les analyses se situant sous ces sections._

**Contexte** :<br>
Il y aurait 467 paramètres sans limite de qualité dans nos données qui ont été identifiés<br>
On veut fournir à Pauline cette liste


**Choses à faire** :
- Sur les données 2024, recroiser avec les catégories de Pauline, et ajouter une colonne à son fichier avec la (ou les) limites trouvées (distinctes).
- Si pas de limite trouvée, laisser vide.
- Notebook qui exporte un fichier Excel


# Résumé de l'étude
***

Je trouve **540 paramètres sans limite de qualité** ( limitequal IS NULL  ) dans la table edc_resultats complète (sans filtre sur l'année)<br>
et 403 paramètres sans limite de qualité en 2024 uniquement.

Ajout des données de limite de qualité dans la table de mapping de Pauline : <br>
J'importe la liste complète dans un fichier excel (**D4G_limite_de_qualite.xlsx**) <br>
On passe de 1612 à 1616 , il y a quelques subtances avec plusieurs valeur de limitequal

# Etude
***
### Packages + read data

In [1]:
# [TO RUN]  Packages
import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

In [2]:
# [TO RUN]
import duckdb
from pipelines.tasks._common import DUCKDB_FILE

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [3]:
# pour pd.to_excel
!uv pip install openpyxl 

Using Python 3.12.9 environment at: C:\Users\Vinca\Documents\D4G_2025\13_pollution_eau\.venv
Audited 1 package in 17ms


###  Exploration

In [4]:
# [OPTIONAL] Tables dispo dans la bdd
con.sql("SHOW TABLES").show()

┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ edc_communes            │
│ edc_prelevements        │
│ edc_resultats           │
│ mapping_categories      │
│ stg_edc__communes       │
│ stg_edc__prevelevements │
│ stg_edc__resultats      │
└─────────────────────────┘



In [5]:
# [OPTIONAL] Preview edc_resultats
preview_resultats = con.sql("SELECT * FROM edc_resultats LIMIT 2").df()
preview_resultats

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
0,001,00100119085,12DCLE,1161,"DICHLOROÉTHANE-1,2","Dichloroéthane-1,2",None,N,L,"<0,50",µg/L,133,<=3 µg/L,None,0.0,107-06-2,00100125759,2020,2025-02-14,20230811-150005
1,001,00100119085,A2H,1832,ATRAZINE-2-HYDROXY,Atrazine-2-hydroxy,None,N,L,"<0,020",µg/L,133,"<=0,1 µg/L",None,0.0,2163-68-0,00100125759,2020,2025-02-14,20230811-150005


In [6]:
# [OPTIONAL] Preview mapping_categories
preview_mapping_categories = con.sql("SELECT * FROM mapping_categories LIMIT 2").df()
preview_mapping_categories

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,PESTOT,6276,TOTAL DES PESTICIDES ANALYSÉS,Total des pesticides analysés,None,pesticides
1,ATRZ,1107,ATRAZINE,Atrazine,1912-24-9,pesticides


In [7]:
con.sql("SELECT COUNT (*) FROM mapping_categories").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         1612 │
└──────────────┘



# Identification des paramètre sans limite de qualité

In [8]:
con.sql(
    "SELECT COUNT(DISTINCT cdparametre ) FROM   edc_resultats WHERE limitequal IS NULL"
).show()

┌─────────────────────────────┐
│ count(DISTINCT cdparametre) │
│            int64            │
├─────────────────────────────┤
│                         540 │
└─────────────────────────────┘



In [9]:
con.sql(
    "SELECT COUNT(DISTINCT cdparametre ) FROM   edc_resultats WHERE limitequal IS NULL AND de_partition='2024'"
).show()

┌─────────────────────────────┐
│ count(DISTINCT cdparametre) │
│            int64            │
├─────────────────────────────┤
│                         403 │
└─────────────────────────────┘



In [10]:
query_no_param_limitequal = """ 
SELECT 
  cdparametre,
  STRING_AGG(DISTINCT libmajparametre) AS list_libmajparametre  ,
  STRING_AGG(DISTINCT cdparametresiseeaux) AS list_cdparametresiseeaux,
  STRING_AGG(DISTINCT casparam) AS list_casparam,
FROM  
    edc_resultats 
WHERE
    limitequal IS NULL 
    AND cdparametre IS NOT NULL
GROUP BY 
    cdparametre
"""

df_no_param_limitequal = con.sql(query_no_param_limitequal).df()
df_no_param_limitequal

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cdparametre,list_libmajparametre,list_cdparametresiseeaux,list_casparam
0,1276,TÉTRACHLORURE DE CARBONE,TCLC,56-23-5
1,1368,"ARGENT,ARGENT DISSOUS","AGD,AG","7440-22-5,7440-22-4"
2,1377,"BÉRYLLIUM,BÉRYLLIUM DISSOUS","BED,BE",7440-41-7
3,5440,BACT. AÉR. REVIVIFIABLES À 22°-68H,GT22_68,None
4,1736,CHLOROMÉTHANE,CLM,74-87-3
...,...,...,...,...
535,2552,SOMME ETHYLTOLUÈNES (3 ISOMÈRES),234ET_S,25550-14-5
536,2690,UNDECANE,UDEC,1120-21-4
537,2094,DALAPON SPD,DAL_SPD,75-99-0
538,6561,"SULFONATE DE PERFLUOROOCTANE,ACIDE SULFONIQUE DE PERFLUOROOCTANE",PFOS,"45298-90-6,1763-23-1"


# Identification des limites de qualité dans les données

In [11]:
# [TO RUN]

query_check_limitequal = """ 
WITH CAT AS (
    SELECT
      *
    FROM
      mapping_categories
),
LIM AS (
SELECT DISTINCT
  cdparametre,
  libmajparametre,
  cdparametresiseeaux,
  casparam,
  limitequal,
  CAST(regexp_extract(REPLACE("limitequal", ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
  regexp_extract("limitequal", '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite
FROM  
    edc_resultats 
)
SELECT 
  CAT.*,
  LIM.limitequal,
  LIM.limitequal_float,
  LIM.unite
FROM  
    CAT 
LEFT JOIN
    LIM
ON 
  LIM.cdparametre =   CAT.cdparametre
  AND LIM.libmajparametre = CAT.libmajparametre
  AND  LIM.cdparametresiseeaux = CAT.cdparametresiseeaux
  AND  LIM.casparam = CAT.casparam
"""

df_check_limitequal = con.sql(query_check_limitequal).df()
df_check_limitequal

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_14964\3641305764.py:3: SyntaxWarning: invalid escape sequence '\d'
  query_check_limitequal = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie,limitequal,limitequal_float,unite
0,CFVP,1464,CHLORFENVINPHOS,Chlorfenvinphos,470-90-6,pesticides,"<=0,1 µg/L",0.1,µg/L
1,CHLFLUA,2950,CHLORFLUAZURON,Chlorfluazuron,71422-67-8,pesticides,"<=0,1 µg/L",0.1,µg/L
2,CHLX,1683,CHLOROXURON,Chloroxuron,1982-47-4,pesticides,"<=0,1 µg/L",0.1,µg/L
3,CLAHB,1757,CHLORDANE BÉTA,Chlordane béta,5103-74-2,pesticides,"<=0,1 µg/L",0.1,µg/L
4,CLAN,1132,CHLORDANE,Chlordane,57-74-9,pesticides,"<=0,1 µg/L",0.1,µg/L
...,...,...,...,...,...,...,...,...,...
1611,RBETA2R,2955,ACTIVITÉ BÉTA GLOB. RÉSIDUELLE BQ/L,Activité béta glob. résiduelle Bq/L,None,radioactivité,None,NaN,None
1612,TACE,<NA>,TAC À L'ÉQUILIBRE,TAC à l'équilibre,None,non classé,None,NaN,None
1613,TRZ_TOT,1282,TRIAZINES,Triazines,None,non classé,None,NaN,None
1614,NEB,1425,NÉBULOSITÉ,Nébulosité,None,non classé,None,NaN,None


On passe de 1612 à 1616 , il y a quelques subtances avec plusieurs valeurs de limitequal

In [12]:
duplicate = df_check_limitequal[
    df_check_limitequal.duplicated(
        ["cdparametre", "libmajparametre", "cdparametresiseeaux", "casparam"]
    )
]
duplicate

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie,limitequal,limitequal_float,unite
190,BMG,1362,BORE MG/L,Bore mg/L,7440-42-8,non classé,None,NaN,None
918,NORFLDM,2737,DESMETHYLNORFLURAZON,Desmethylnorflurazon,23576-24-1,métabolite de pesticide,"<=0,1 µg/L",0.1,µg/L
1039,SE,1385,SÉLÉNIUM,Sélénium,7782-49-2,métaux lourds,None,NaN,None
1061,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),14797-65-0,nitrite,"<=0,5 mg/L",0.5,mg/L
1393,CYANO53,<NA>,JAAGINEMA SP,Jaaginema sp,None,microbio,None,NaN,None


In [13]:
list_of_values = duplicate["cdparametresiseeaux"]

df_check_limitequal[
    df_check_limitequal["cdparametresiseeaux"].isin(list_of_values)
].sort_values(by=["cdparametre"])

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie,limitequal,limitequal_float,unite
784,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),14797-65-0,nitrite,"<=0,1 mg/L",0.1,mg/L
1061,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),14797-65-0,nitrite,"<=0,5 mg/L",0.5,mg/L
1580,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),None,nitrite,None,NaN,None
127,BMG,1362,BORE MG/L,Bore mg/L,7440-42-8,non classé,"<=1,5 mg/L",1.5,mg/L
190,BMG,1362,BORE MG/L,Bore mg/L,7440-42-8,non classé,None,NaN,None
658,SE,1385,SÉLÉNIUM,Sélénium,7782-49-2,métaux lourds,<=20 µg/L,20.0,µg/L
1039,SE,1385,SÉLÉNIUM,Sélénium,7782-49-2,métaux lourds,None,NaN,None
766,NORFLDM,2737,DESMETHYLNORFLURAZON,Desmethylnorflurazon,23576-24-1,métabolite de pesticide,None,NaN,None
918,NORFLDM,2737,DESMETHYLNORFLURAZON,Desmethylnorflurazon,23576-24-1,métabolite de pesticide,"<=0,1 µg/L",0.1,µg/L
1392,CYANO53,<NA>,JAAGINEMA SP,None,None,microbio,None,NaN,None


In [14]:
# To excel
df_check_limitequal.to_excel("D4G_limite_de_qualite.xlsx")